We shouldn't turn on workflows 2a, 2b, and 2c until this workflow 1 final subject set is complete. 

Here is a list of steps that need to be carried out in order to be ready to activate workflows 2a, 2b, and 2c. 

-  Determine the final, aggregated consensus results for the subjects (the journal pages) that have been processed through workflow 1. 

-  Use these consensus results to create 3 new subject sets: 

   - subject set 1) images that should be associated w/ workflow 2a -- i.e., pages with just 1 single sky figure with axes labeled

   - subject set 2) images that should be associated w/ workflow 2b -- i.e., pages with multiple sky figures with axes labeled

   - subject set 3) images that should be associated w/ workflow 2c -- i.e., pages with sky figures without axes labeled

Once you have access to the Project Editor for Astronomy Rewind, you can click on 'Workflows' to see workflows 2a, 2b, and 2c are. Once you go into the editor for each workflow, you can click on 'Test this workflow' to see what the volunteer experience will be in answering the tasks/questions for that workflow.

Note: with this many images, it's best to use one of our API clients to automate the upload of subjects into a new subject set. See https://github.com/zooniverse/panoptes-python-client or https://github.com/zooniverse/panoptes-cli

- Associate the new subject sets with their respective workflow (under 'associated subject set' in the project editor for that workflow). Also de-associate the current 'test images' subject sets that are just used for testing. 

- Once the subject sets are associated, have Alyssa, Julie and Gretchen review each workflow to make sure all the help context, text, etc. is correct and to make sure that the experience into WWT is still working properly. 

- Once workflow 1 is finished, go to the 'Visibility' tab and make Workflows 2a, 2b, and 2c active. 


In [4]:
import os
import json

import pandas as pd

from panoptes_client import Panoptes, SubjectSet
# https://github.com/zooniverse/panoptes-python-client

In [5]:
def JSONParser(data):
    """call json.loads"""
    return json.loads(data)


def load_classifications(filename, json_columns=None):
    """
    Load classifications into pandas dataframe.
    
    Some columns of the csv are embedded json and need special parsing.
    """
    json_columns = json_columns or ['metadata', 'annotations', 'subject_data']
    converters = {i: JSONParser for i in json_columns}

    return pd.read_csv(filename, converters=converters)


def unpack(series):
    """
    Return the first value in a series.

    All annotations values are lists because of a few multiple tasks.
    The second multiple task always has the value of 'None of the above'
    (For this dataset!)
    """
    return [a[0] for a in series]


def parse_classifications(filename):
    """
    Load classifications and datamunge annotations column.
    """
    data = load_classifications(filename)

    # Only need the first item in the annotations list of json objects
    data['annotations'] = unpack(data['annotations'])
    return data


def explore(data):
    """ 
    print the values are in the annotations
    """
    import numpy as np
    values = np.unique(np.concatenate([a['value'] for a in data['annotations']]))
    print(values)
    return


def cull_subject_ids(filename, w2s=None, overwrite=False, add=True):
    """
    Cull classifications file and write to new workflows.
    
    Parameters
    ----------
    filename : str
        input csv file
    
    ws2 : dict
        key: workflow title (for file naming)
        value: Single string to select on within annotations values.
    
    overwrite : bool [False]
        overwrite with new file 
    """
    # load and munge data
    data = parse_classifications(filename)
    
    if w2s is None:
        # see, e.g., explore(data)
        w2s = {'2a': 'A single sky\xa0figure *with* axes labeled',
               '2b': 'Two or more sky figures *with* axes labeled',
               '2c': 'Sky figure(s) *without* axes labeled'}
        # Empty set IDs created on the web interface
        subject_set_ids = {'2a': 12651,
                           '2b': 8433,
                           '2c': 12645}     

    for wf in w2s.keys():
        # new filename assumes wf1 is in the first filename!
        outname = filename.replace('classifications_wf1', 'subject_ids_wf{0:s}'.format(wf))
        
        # Identifiy matches to next workflow
        iwf = [w2s[wf] in a['value'] for a in data['annotations']]

        # create sub-copy of dataframe with only workflow matches
        df = data['subject_ids'].iloc[iwf]

        # write ... or not
        if not os.path.isfile(outname) or overwrite:
            df.to_csv(outname, index=False)
            msg = 'wrote'
            if add:
                add_to_subject_set(subject_set_ids['wf'], outname)
        else:
            msg = 'not overwriting'
        print('{0:s} {1:s}'.format(msg, outname))

    return

    
def add_to_subject_set(subject_set_id, subject_set_file)
    subject_set = SubjectSet.find(subject_set_id)
    with open(subject_set_file) as subject_ids:
        subject_set.add(subject_ids.read().splitlines())
    return


def cull_wf1(classifications_filename, wf1_filename, overwrite=False):
    if not os.path.isfile(wf1_filename) or overwrite:
        # Slice out all other workflows and testing (and maintain the header line)
        os.system('head -1 {0:s} > {1:s}'.format(classifications_filename, wf1_filename))
        os.system('grep "Workflow 1: Identifying figure types" {0:s} >> {1:s}'.format(classifications_filename, wf1_filename))
    return

def main(argv=None):
    # log in.
    parser = argparse.ArgumentParser(description="Add subject sets to workflow 2a,b,c from classifications")

    parser.add_argument('username', type=str,
                        help='zooniverse username')

    parser.add_argument('password', type=str,
                        help='password')
    
    args = parse_args(argv)
    
    Panoptes.connect(username=args.username, password=args.password)
    
    classifications_filename = 'astronomy-rewind-classifications.csv'
    wf1_filename = 'astronomy-rewind-classifications_wf1.csv'

    cull_wf1(classifications_filename, wf1_filename))
    cull_subject_ids(wf1_filename)


if __name__ == "__main__":
    sys.exit(main())


wrote astronomy-rewind-subject_ids_wf2a.csv
wrote astronomy-rewind-subject_ids_wf2b.csv
wrote astronomy-rewind-subject_ids_wf2c.csv
